Reads and formats the test data

In [67]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

test_data = pd.read_csv('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/training data/training_data.csv')
test_data.fillna('0', inplace=True)

conversion_dict = {
    'PAYER': str,
    'Process': str,
    'SHS Status': str,
    'PIC Desc': str,
    'ACK': int,
    'Form Letter': int,
    'Records': int,
    'Status': str,
    'Category': str,
    'Comment': str
}
test_data = test_data.astype(conversion_dict)

Sets up the label encoders and training dataframe

In [68]:
le_payer = LabelEncoder()
le_process = LabelEncoder()
le_shs_status = LabelEncoder()
le_pic_desc = LabelEncoder()

le_status = LabelEncoder()
le_category = LabelEncoder()
le_comment = LabelEncoder()

test_data['payer_n'] = le_payer.fit_transform(test_data['PAYER'])
np.save('../ML Training/classes/payer_classes.npy', le_payer.classes_)
test_data['process_n'] = le_process.fit_transform(test_data['Process'])
np.save('../ML Training/classes/process_classes.npy', le_process.classes_)
test_data['shs_status_n'] = le_shs_status.fit_transform(test_data['SHS Status'])
np.save('../ML Training/classes/shs_status_classes.npy', le_shs_status.classes_)
test_data['pic_desc_n'] = le_pic_desc.fit_transform(test_data['PIC Desc'])
np.save('../ML Training/classes/pic_desc_classes.npy', le_pic_desc.classes_)

test_data['status_n'] = le_status.fit_transform(test_data['Status'])
np.save('../ML Training/classes/status_classes.npy', le_status.classes_)
test_data['category_n'] = le_category.fit_transform(test_data['Category'])
np.save('../ML Training/classes/category_classes.npy', le_category.classes_)
test_data['comment_n'] = le_comment.fit_transform(test_data['Comment'])
np.save('../ML Training/classes/comment_classes.npy', le_comment.classes_)

training_data = test_data.copy()
copy = training_data.copy()

In [69]:
training_data.drop(training_data.loc[:, 'PAYER':'PIC Desc'], axis='columns', inplace=True)
training_data.drop(training_data.loc[:, 'Status':'Comment'], axis='columns', inplace=True)
training_data.drop(training_data.loc[:, 'status_n':'comment_n'], axis='columns', inplace=True)

X = training_data
y = pd.DataFrame(test_data['status_n'])
Status = DecisionTreeClassifier()

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
Status.fit(X_train, y_train)
predictions = Status.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy*100

99.62732919254658

In [71]:
import joblib
joblib.dump(Status, './models/status_model.joblib')

['./models/status_model.joblib']

In [72]:
training_data = copy.copy()

In [73]:
y = pd.DataFrame(training_data['category_n'])

training_data.drop(training_data.loc[:, 'PAYER':'PIC Desc'], axis='columns', inplace=True)
training_data.drop(training_data.loc[:, 'Status':'Comment'], axis='columns', inplace=True)
training_data.drop(training_data.loc[:, 'category_n':'comment_n'], axis='columns', inplace=True)

X = training_data

Category = DecisionTreeClassifier()

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
Category.fit(X_train, y_train)
predictions = Category.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy*100

99.92546583850933

In [75]:
joblib.dump(Category, './models/category_model.joblib')

['./models/category_model.joblib']

In [76]:
training_data = copy.copy()

In [77]:
y = pd.DataFrame(training_data['comment_n'])

training_data.drop(training_data.loc[:, 'PAYER':'PIC Desc'], axis='columns', inplace=True)
training_data.drop(training_data.loc[:, 'Status':'Comment'], axis='columns', inplace=True)
training_data.drop(['comment_n'], axis='columns', inplace=True)

X = training_data

Category = DecisionTreeClassifier()

In [78]:
X = training_data

Comment = DecisionTreeClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
Comment.fit(X_train, y_train)
predictions = Comment.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy * 100

99.95031055900621

In [79]:
joblib.dump(Comment, './models/comment_model.joblib')

['./models/comment_model.joblib']